In [1]:
import numpy as np
import pandas as pd
import ast
import nltk
from nltk.stem.porter import PorterStemmer

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [4]:
movies = movies.merge(credits,on = "title")

In [5]:
movies = movies[['id','title','overview','genres','keywords','cast','crew']]

In [6]:
movies.isnull().sum()

id          0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [7]:
movies.dropna(inplace=True)

In [8]:
movies.duplicated().sum()

0

In [9]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [10]:
def convert(object):
    newList = []
    for i in ast.literal_eval(object):
        newList.append(i["name"])
    return newList


In [11]:
movies["genres"]=movies["genres"].apply(convert)

In [12]:
movies["keywords"] = movies["keywords"].apply(convert)

In [13]:

def convertcast(object):
    newList = []
    count = 0
    for i in ast.literal_eval(object):
        if count!=3:
            newList.append(i["name"])
            count+=1
        else:
            break
    return newList



In [14]:
movies["cast"] = movies["cast"].apply(convertcast)

In [15]:
def convertcrew(object):
    newList = []
    for i in ast.literal_eval(object):
        if i["job"]=='Director':
            newList.append(i["name"])
            break
    return newList

In [16]:
movies['crew'] = movies['crew'].apply(convertcrew)


In [17]:
movies['overview'].apply (lambda x:x.split())

0       [In, the, 22nd, century,, a, paraplegic, Marin...
1       [Captain, Barbossa,, long, believed, to, be, d...
2       [A, cryptic, message, from, Bond’s, past, send...
3       [Following, the, death, of, District, Attorney...
4       [John, Carter, is, a, war-weary,, former, mili...
                              ...                        
4804    [El, Mariachi, just, wants, to, play, his, gui...
4805    [A, newlywed, couple's, honeymoon, is, upended...
4806    ["Signed,, Sealed,, Delivered", introduces, a,...
4807    [When, ambitious, New, York, attorney, Sam, is...
4808    [Ever, since, the, second, grade, when, he, fi...
Name: overview, Length: 4806, dtype: object

In [18]:
movies['genres'] = movies['genres'].apply (lambda x:[i.replace(" ","")for i in x])
movies['keywords'] = movies['keywords'].apply (lambda x:[i.replace(" ","")for i in x])
movies['cast'] = movies['cast'].apply (lambda x:[i.replace(" ","")for i in x])
movies['crew'] = movies['crew'].apply (lambda x:[i.replace(" ","")for i in x])
movies["overview"] = movies["overview"].apply (lambda x:x.split())


In [19]:
movies['tags'] = movies["overview"] + movies["genres"] + movies["keywords"] + movies["cast"] + movies["crew"]

In [20]:
new_df = movies[['id','title','tags']]

In [21]:
new_df["tags"] = new_df["tags"].apply(lambda x:" ".join(x))

C:\Users\Nabaraj\AppData\Local\Temp\ipykernel_2092\1578042357.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"] = new_df["tags"].apply(lambda x:" ".join(x))


In [22]:
new_df['tags'] = new_df['tags'].apply (lambda x:x.lower())

C:\Users\Nabaraj\AppData\Local\Temp\ipykernel_2092\645746771.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply (lambda x:x.lower())


In [23]:
import nltk
from nltk.stem.porter import PorterStemmer

In [24]:
ps = PorterStemmer()

In [25]:
def stem(text):
    y = []
    for i in text.split():
       y.append(ps.stem(i))
    return " ".join(y)


In [26]:
new_df['tags']=new_df['tags'].apply(stem)

C:\Users\Nabaraj\AppData\Local\Temp\ipykernel_2092\3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [27]:
new_df['tags'][0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english',max_features=2000)

In [29]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [30]:
cv.get_feature_names_out()

array(['000', '10', '11', ..., 'youth', 'zombi', 'zoo'], dtype=object)

In [31]:
vectors.shape

(4806, 2000)

In [32]:
def cosine_similarity(x, y):
    dot_product = np.dot(x, y)
    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)
    return dot_product / (norm_x * norm_y)



In [33]:
n_movies = vectors.shape[0]
similarity = np.zeros((n_movies, n_movies))

In [34]:
for i in range(n_movies):
    for j in range(n_movies):
        similarity[i][j] = cosine_similarity(vectors[i], vectors[j])


In [35]:
from fuzzywuzzy import process

C:\Python\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [36]:
# from sklearn.neighbors import NearestNeighbors
# model = NearestNeighbors(n_neighbors = 10,metric='cosine').fit(similarity)


In [37]:
def recommender(movie):
    idx = process.extractOne(movie, movies['title'])[2]
    print('Movie Selected: ', movies['title'][idx], 'Index: ', idx)
    print('Searching for recommendations...')
    distances = similarity[idx]
    indices = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:11]
    for i in indices:
        print(movies.iloc[i[0]].title)


In [41]:
recommender('fight club')

Movie Selected:  Fight Club Index:  662
Searching for recommendations...
This Is Martin Bonner
Fight Valley
Northfork
Paint Your Wagon
Rudderless
Death Race
Never Back Down
The Master
Snow Angels
Tumbleweeds
